# <font color = "red">Classificação de Risco de Crédito</font>

## Uma análise Comparativa Para Prever Inadimplência 

<font color = "red">Problema de Negócio:</font> A partir de dados históricos, irei verificar os padrões que levaram os clientes de uma instituição financeira, a adentrarem, ou não, na situação de inadimplência, para posteriormente criar um 

In [1]:
# Imports 

import pandas as pd
import numpy as np

In [6]:
df_original = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/risco_credito_original.csv")

In [8]:
df_original.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [10]:
df_original.shape

(32581, 12)

In [11]:
df_original.sample(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
19390,28,31000,RENT,2.0,PERSONAL,B,3000,10.00,0,0.10,N,7
16733,23,47000,MORTGAGE,7.0,EDUCATION,C,13850,12.68,0,0.29,Y,2
32439,55,51000,RENT,2.0,PERSONAL,A,8000,5.79,0,0.16,N,26
3803,22,30000,RENT,1.0,PERSONAL,C,4000,NaN,0,0.13,N,4
31483,41,98004,MORTGAGE,25.0,DEBTCONSOLIDATION,C,12000,13.57,0,0.12,N,15
29438,37,85000,RENT,21.0,VENTURE,A,1200,8.90,0,0.01,N,13
7147,26,67550,RENT,3.0,MEDICAL,A,7000,8.90,0,0.10,N,2
4056,24,70000,RENT,8.0,VENTURE,C,4000,13.49,0,0.06,Y,4
25087,27,80000,MORTGAGE,3.0,DEBTCONSOLIDATION,B,3000,11.11,0,0.04,N,7
15352,22,130000,MORTGAGE,2.0,EDUCATION,B,13000,11.14,0,0.10,N,3


In [13]:
df_original.loan_status.value_counts()

loan_status
0    25473
1     7108
Name: count, dtype: int64

In [ ]:
personagem	Idade
pessoa_renda	Renda Anual
pessoa_propriedade_de_casa	Propriedade da casa
pessoa_emp_length	Duração do emprego (em anos)
intenção_de_empréstimo	Intenção de empréstimo
grau_de_empréstimo	Grau de empréstimo
valor_emprestado	Valor do empréstimo
taxa_interna_empréstimo	Taxa de juro
status_do_empréstimo	Status do empréstimo (0 é não inadimplente, 1 é inadimplente)
renda_porcentagem_empréstimo	Renda percentual
cb_pessoa_padrão_em_arquivo	Inadimplência histórica
cb_preson_cred_hist_length	Duração do histórico de crédito